In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue May  2 19:44:01 2023

@author: Jordanius

KGG370 Analysis of Observations

Assessment 2: Least Squares Adjustments

"""

# First import the required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math as math
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error


# Define the required functions

def Join(E1, N1, E2, N2):
    """Computes the bearing and distance between two known points

    Parameters
    ----------
    E1 : float
        The Easting of the first point
    N1 : float
        The Northing of the first point
    E2 : float
        The Easting of the second point
    N2 : float
        The Northing of the second point

    Returns
    -------
    D : float
        The distance between point 1 and 2
    B : float
        The bearing from point 1 to 2 (in radians)
    """
    dE = E2 - E1 
    dN = N2 - N1
    if dE > 0 and dN < 0 or dE < 0 and dN < 0:
        adder = np.pi
    elif dE < 0 and dN > 0:
        adder = 2*np.pi
    elif dE > 0 and dN > 0:
        adder = 0
    D = np.sqrt(dE**2 + dN**2)
    B = np.arctan(dE/dN) + adder
    
    return D, B


def rad_to_dms(rad):
    'Adaptation of Luke Wallace''s function'
    """Converts an angle in radians to an angle 
       in degrees, minutes, seconds

    Parameters
    ----------
    rad : float
          An angle in radians

    Returns
    -------
    dms : tuple
          The angle in degrees, minutes and seconds
  
    """
    dd = np.rad2deg(rad)
    d = math.floor(dd)
    m = math.floor((dd-d)*60)
    s = (dd - d - m/60)*3600
    dms = tuple([d,m,s])
    
    return dms

def Radiation(d, B12, E1, N1):
    """Computes the coordinates of an unknown point given
    a known point's coordinates, and a distance and bearing
    between those points
    

    Parameters
    ----------
    d : float
        Observed distance between point 1 & 2
    B12 : float
        The bearing from point 1 to 2
    E2 : float
        The Easting of known point 1
    N2 : float
        The Northing of knwon point 1

    Returns
    -------
    E2 : float
        The Easting of point 2
    N2 : float
        The Northing of point 2
    """
    
    dE = d*np.sin(B12)
    dN = d*np.cos(B12)
    E2 = round(dE + E1, 4)
    N2 = round(dN + N1, 4)
    
    return E2, N2

#### Function doesn't handle negative degrees properly
def dms_radians(dms_string):
    """Converts a degrees-minutes-seconds string into radians

    Parameters
    ----------
    dms_string : string
               An angle degrees-minutes-seconds format

    Returns
    -------
    rad : float
        Angle converted to radians
  
    """
    degrees = sum(np.fromstring(dms_string, sep = '-') * [1.0, 1/60.0, 1/3600.0])
    rad = np.radians(degrees)
    
    return rad

def fudger(a, b):
    """Computes the 95% horizontal circular confidence region
    given semi-major and semi-minor axes of 1-sigma error ellipse

    Parameters
    ----------
    a : float
      semi-major axis of 1-sigma error ellipse
    b : float
      semi-minor axis of 1-sigma error ellipse

    Returns
    -------
    r : float
      Radius of 95% horizontal circular confidence region 
  
    """
    q0 = 1.960790
    q1 = 0.004071
    q2 = 0.114276
    q3 = 0.371625
    C = b/a
    K = q0 + q1*C + q2*(C**2) + q3*(C**3)
    r = a * K
    
    return r

#Define the exponential function
def exponential_func(x, a, b, c):
    """Computes the output of the exponential function model

    Parameters
    ----------
    x : float
        Independent variable
    a : float
        Constant
    b : float
        Constant
    c : float
        Constant

    Returns
    -------
    Model : float
          The output value from the exponential function
    """
    model =  a * np.exp(-b * x) + c
    
    return model



# Define a dictionary of points 1-6 and their corresponding Eastings and Northings
coords = {'Point': [1,2,3,4,5,6], 'Easting [m] (1)': [515.6680, 417.9720, 579.2641, 
                                                      563.5981, 735.9475, 765.4379],
          'Northing [m] (1)': [415.7430, 631.4860, 587.1696, 779.5893, 554.5154, 745.7677], 
          'Easting [m] (2)': [515.6680, 417.9720, 579.2647, 563.5733, 735.9405, 765.4158],
          'Northing [m] (2)': [415.7430, 631.4860, 587.2078, 779.6182, 554.5567, 745.8130]}

# Create a dataframe from the dictionary created above
coord_dat = pd.DataFrame(data=coords)



##### Task 3.4 - Find HD, SD and change in height from stn 25 to 225 #####

# Define a dictionary of points 25 & 225 and their corresponding Eastings and Northings
coords = {'Point': [25,225], 'Easting [m]': [99.9275, 99.9274],
          'Northing [m]': [987.5985, 987.5993], 
          'Height [m]': [10.0878, 10.0873]}

# Create a dataframe from the dictionary created above
stn25_225 = pd.DataFrame(data=coords)

# Compute the join between points 25 & 225
joiner = Join(stn25_225.loc[0,'Easting [m]'],stn25_225.loc[0,'Northing [m]'],stn25_225.loc[1,'Easting [m]'],stn25_225.loc[1,'Northing [m]'] )

# Extract the bearing from point 25 to 225
brg = rad_to_dms(joiner[1])

# Extract the horizontal distance between points 25 & 225
HD = joiner[0]

# Compute the change in height between points 25 & 225
dH = stn25_225.loc[1, 'Height [m]'] - stn25_225.loc[0, 'Height [m]'] 

# Compute the change in Easting between points 25 & 225
dE = stn25_225.loc[1, 'Easting [m]'] - stn25_225.loc[0, 'Easting [m]']

# Compute the change in Northing between points 25 & 225
dN = stn25_225.loc[1, 'Northing [m]'] - stn25_225.loc[0, 'Northing [m]']

# Compute the slope distance between points 25 & 225
SD = np.sqrt(dH**2 + HD**2)
SD = round(SD, 4)


# Define the Jacobian matrix
Jacobian = np.array([[-1, 1, 0, 0, 0, 0],
                    [ 0, 0, -1, 1, 0, 0],
                    [ 0, 0, 0, 0, -1, 1]])

# Define the variance-covariance matrix (VCV)
VCV = np.array([
    [0.027, 0.016, 0.007, 0.002, 0.000, 0.000],
    [0.016, 0.138, 0.003, 0.056, 0.000, 0.001],     
    [0.007, 0.003, 0.039, 0.024, 0.001, 0.000],
    [0.002, 0.056, 0.024, 0.061, 0.000, 0.001],
    [0.000, 0.000, 0.001, 0.000, 0.014, 0.004],
    [0.000, 0.001, 0.000, 0.001, 0.004, 0.025]
])

# Compute the product: Jacobian * VCV * Jacobian.T
propagated_VCV = np.matmul(np.matmul(Jacobian, VCV), Jacobian.T)

print("Propagated Variance-Covariance Matrix:")
print(propagated_VCV)

# Compute the standard deviation of change in height
sdH = np.sqrt(propagated_VCV[2][2])
print('Standard deviation of sdH: ')
print(sdH)

# Define the Jacobian matrix
Jacobian = np.array([dE/SD, dN/SD, dH/SD])

# Define the variance-covariance matrix (VCV)
VCV = propagated_VCV

# Compute the product: Jacobian * VCV * Jacobian.T
propagated_VCV_SD = np.matmul(np.matmul(Jacobian, VCV), Jacobian.T)

# Compute the standard deviation of slope distance
sSD = np.sqrt(propagated_VCV_SD)
print('Standard deviation of SD: ')
print(sSD)

# Define the Jacobian matrix
Jacobian = np.array([dE/HD, dN/HD, dH/HD])

# Define the variance-covariance matrix (VCV)
VCV = propagated_VCV

# Compute the product: Jacobian * VCV * Jacobian.T
propagated_VCV = np.matmul(np.matmul(Jacobian, VCV), Jacobian.T)

# Compute the standard deviation of horizontal distance
sHD = np.sqrt(propagated_VCV)
print('Standard deviation of HD: ')
print(sHD)


###### Task 4 #######

# Define a dictionary with the semi-major and semi-minor axes for points 3-6
adj_coords = {'Point': [3,4,5,6], 'Semi-Major a [mm]': [15.08, 23.41, 26.94, 33.42],
          'Semi-minor b [mm]': [9.63, 9.67, 9.70, 9.83], 
          }

# Create a new dataframe from adjusted coordinate ellipse data
adjcoor = pd.DataFrame(data=adj_coords)

# Define an empty list to fill with computed Positional Uncertainty values
PU = [0,0,0,0]

# Compute the Positional Uncertainty for each point using the fudger function
for i in range(len(adjcoor)):
    PU[i] = fudger(adjcoor.loc[i, 'Semi-Major a [mm]'], adjcoor.loc[i, 'Semi-minor b [mm]'])

adjcoor['PU [mm]'] = PU

# Write dataframe to CSV file
adjcoor.to_csv('/Users/Jordanius/Desktop/UTAS/KGG370 - ANOBS/Task 4 PU.csv', index = False)


### Task 4.3 ####
# Convert bearing to radians
brg = dms_radians('335-38-14.3')


# Distance reduced by a factor of 2
d = 157.888

# Compute Easting and Northing 
e, n = Radiation(d, brg, 515.6680, 415.7430)
print(f'Distance reduced by a factor of 1.5\nEasting: {e} m\nNorthing: {n} m')

# Distance reduced by a factor of 2
d = 118.416

# Compute Easting and Northing
e, n = Radiation(d, brg, 515.6680, 415.7430)
print(f'Distance reduced by a factor of 2\nEasting: {e} m\nNorthing: {n} m')


#### Task 4.3 c) #####

# Distance 2km
d = 2000
# Southwest bearing (225 degrees)
brg = dms_radians('225-0-0')
# Compute Easting & Northing of point 7
e7, n7 = Radiation(d, brg, 515.668, 415.743)

# Semi-major and semi-minor axes for points 3-6 with backsight reduced by factor of 1.5
factor1 = [[19.54, 37.91, 36.40, 49.88],
[9.61, 9.66, 9.69, 9.83]]

# Compute PU for points 3-6 with backsight length reduced by factor of 1.5
for i in range(len(factor1[0])):
    fudge = fudger(factor1[0][i], factor1[1][i])
    print(f'\n PU for Point {i + 3} = {fudge} mm') 
    
# Semi-major and semi-minor axes for points 3-6 with backsight reduced by factor of 2
factor2 = [[25.75, 53.23, 46.67, 67.13],[9.60, 9.67, 9.69, 9.84]]

# Compute PU for points 3-6 with backsight length reduced by factor of 2
for i in range(len(factor2[0])):
    fudge = fudger(factor2[0][i], factor2[1][i])
    print(f'\n PU for Point {i + 3} = {fudge} mm')

    # Semi-major and semi-minor axes for points 3-6 with 2km backsight
factor3 = [[10.72, 12.63, 13.23, 15.24],[9.61, 9.67, 9.65, 9.82]]

# Compute PU for points 3-6 with 2km backsight in survey network
for i in range(len(factor3[0])):
    fudge = fudger(factor3[0][i], factor3[1][i])
    print(f'\n PU for Point {i + 3} = {fudge} mm')


#### Code for plot of point 6 data #####
# Input data
dist = np.array([118.416, 157.888, 238.834, 2000.000])
PU = np.array([131.9, 98.2, 66.2, 32.2])

# Normalize the data
dist_normalized = dist / 1000.0
PU_normalized = PU / 100.0

# Provide adjusted initial parameter values
initial_params = [max(PU_normalized), -0.001, min(PU_normalized)]

# Fit the exponential model
params, params_covariance = curve_fit(exponential_func, dist_normalized, PU_normalized, p0=initial_params)

# Extract the parameters
a, b, c = params
# Find the distance value where the model equals 100

# Calculate the predicted values
dist_predicted = dist
PU_predicted = exponential_func(dist, a, b, c)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(PU, PU_predicted))

# Generate points on the fitted curve
x_fit = np.linspace(min(dist_normalized), max(dist_normalized), 100)
y_fit = exponential_func(x_fit, a, b, c)
plt.figure(figsize=(8, 6))

# Plot the data points and fitted curve
plt.plot(x_fit * 1000.0, y_fit * 100.0, color = 'lightgreen', label='Modelled trend')
plt.scatter(dist, PU, label='Point 6 data', c = 'indigo')
plt.xlabel('Backsight Distance [m]')
plt.ylabel('PU [mm]')
plt.title('Point 6: Positional Uncertainty versus backsight distance', fontweight = 'bold')
plt.legend()
plt.show()
disty_100 = np.log(a/(100-c))/-b


plt.show()




